In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import time
from collections import Counter

# Load the saved model
model = load_model('emotion_detection_model.h5')

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Emotion-to-Playlist Mapping
emotion_to_playlist = {
    'Angry': 'https://open.spotify.com/playlist/37i9dQZF1EIgNZCaOGb0Mi?si=a4843785407d4eb1',  
    'Disgust': 'https://open.spotify.com/playlist/37i9dQZF1EId2IRKziWcgn?si=beabff143f59433a',  
    'Fear': 'https://open.spotify.com/playlist/37i9dQZF1EIfTmpqlGn32s?si=06e9e18e2c914398', 
    'Happy': 'https://open.spotify.com/playlist/37i9dQZF1EVJSvZp5AOML2?si=8860870b4ca24f43',  
    'Neutral': 'https://open.spotify.com/playlist/37i9dQZF1EIcJuX6lvhrpW?si=05d425fe1a3749e3', 
    'Sad': 'https://open.spotify.com/playlist/37i9dQZF1EVKuMoAJjoTIw?si=31c9ee5c12ff4a91',  
    'Surprise': 'https://open.spotify.com/playlist/37i9dQZF1DWYWddJiPzbvb?si=52b14ddf486c47df' 
}

# Initialize webcam
cap = cv2.VideoCapture(0)

# Track detected emotions
detected_emotions = []

# Start the timer
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi = gray_frame[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (48, 48)) / 255.0
        roi_reshaped = np.reshape(roi_resized, (1, 48, 48, 1))
        
        # Predict emotion
        prediction = model.predict(roi_reshaped)
        emotion = emotion_labels[np.argmax(prediction)]
        detected_emotions.append(emotion)  # Save the detected emotion

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Real-Time Emotion Detection', frame)
    
    # Stop after 5 seconds
    if time.time() - start_time > 30:
        break

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Determine the most frequently detected emotion
if detected_emotions:
    emotion_count = Counter(detected_emotions)
    most_frequent_emotion = emotion_count.most_common(1)[0][0]
    print(f"Detected Emotion: {most_frequent_emotion}")
    
    # Retrieve and print the associated playlist
    playlist_url = emotion_to_playlist.get(most_frequent_emotion, "No playlist available")
    print(f"Associated Playlist: {playlist_url}")
else:
    print("No emotion detected.")


1/1 [==============================] - 0s 20ms/step
Detected Emotion: Happy
Associated Playlist: https://open.spotify.com/playlist/37i9dQZF1EVJSvZp5AOML2?si=8860870b4ca24f43
